In [ ]:
from blazingsql import BlazingContext
bc = BlazingContext()

import pandas as pd
from sqlalchemy import create_engine
import urllib
import json
import datetime

In [ ]:
from dotenv_config import Config
config = Config('.env')

In [ ]:
params = urllib.parse.quote_plus(f"DRIVER={{ODBC Driver 17 for SQL Server}};"
                                 f"SERVER={config('SERVER')};"
                                 f"DATABASE={config('DATABASE')};"
                                 f"Uid={config('Uid')};"
                                 f"Pwd={config('Pwd')};")

In [ ]:
# Connect via current Windows credentials
conn = create_engine("mssql+pyodbc:///?odbc_connect={}".format(params))

In [ ]:
table_name = "CSPPriceList"

In [ ]:
df = pd.read_sql(f"SELECT * FROM {table_name} ORDER BY [Offer ID], Month", conn)

In [ ]:
df.rename( columns= {
    'A C D U': 'A_C_D_U', 
    'Valid From Date': 'Valid_From_Date', 
    'Valid To Date': 'Valid_To_Date', 
    'Offer Display Name': 'Offer_Display_Name', 
    'Offer ID': 'Offer_ID', 
    'License Agreement Type': 'License_Agreement_Type', 
    'Purchase Unit': 'Purchase_Unit', 
    'Secondary License Type': 'Secondary_License_Type', 
    'End Customer Type': 'End_Customer_Type', 
    'List Price': 'List_Price', 
    'ERP Price': 'ERP_Price', 
    'Month': 'MonthKey'
    }, inplace=True)

In [ ]:
df.info()

In [ ]:
bc.create_table('csp_pricelist', df)

In [ ]:
# running the query will give you a token object
token = bc.sql('SELECT * FROM main.csp_pricelist ', ['csp_pricelist']) 
# with the token object you can retrieve your results
result = token.get()
# the gdf will be in the result object
gdf = result.columns
print(gdf)

In [ ]:
gdf['Offer_Hash'] = gdf.hash_columns(['A_C_D_U',
'Offer_Display_Name',
'License_Agreement_Type',
'Purchase_Unit',
'Secondary_License_Type',
'End_Customer_Type',
'List_Price',
'ERP_Price'])

In [ ]:
dedup = gdf.to_pandas().drop_duplicates(subset=['Offer_ID', 'Offer_Hash'], keep='first')

In [ ]:
dedup.count()

In [ ]:
dedup.head(10)

In [ ]:
dedup.to_sql(name="TestTable", con=conn, index=False, if_exists="append")

In [ ]:
offer = '021c86b6-8f1d-4317-bcca-1a55ee5377d8'
dedup.query('Offer_ID == @offer')